LSTM MODEL

In [1]:
get_ipython().system('pip install sentence-transformers pandas pinecone-client')

Defaulting to user installation because normal site-packages is not writeable


In [2]:
get_ipython().system('pip install yfinance')

Defaulting to user installation because normal site-packages is not writeable


In [3]:
get_ipython().system('pip install tf-keras')

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import logging
import yfinance as yf

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [6]:
# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Pinecone
pc = Pinecone(api_key="09bb980b-6cef-48c3-9aa5-63f3cbc9885e")
index = pc.Index("financial-data-index")

# Initialize SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
logger.info("SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.")

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['C:\\Users\\Pham Ty\\AppData\\Roaming\\Python\\Python312\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
INFO:__main__:SentenceTransformer 'all-MiniLM-L6-v2' loaded suc

In [18]:
# Folder containing the CSV files
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'

# Folder to save the processed CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'

In [19]:
# Tạo thư mục nếu không tồn tại
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [20]:
# Định nghĩa danh sách các mã chứng khoán
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
    'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
    'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
    'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
    'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
    'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
    'CMG', 'BCS', 'UAA']

In [21]:
def fetch_and_save_stock_data(symbol, start_date='2014-09-18', end_date='2024-09-18'):
    """
    Fetch stock data from Yahoo Finance, save to CSV, and upsert data into Pinecone.
    """
    try:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        
        # Save data to CSV
        csv_file_path = os.path.join(folder_path, f'{symbol}.csv')
        stock_data.to_csv(csv_file_path)
        logger.info(f"Data for {symbol} saved to {csv_file_path}")
        
        # Load CSV data
        df = pd.read_csv(csv_file_path)
        
        # Select necessary columns and drop NaN values
        required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"{csv_file_path} is missing one or more required columns: {required_columns}")
            return
        
        df = df[required_columns].dropna()

        # Convert each row into a single string of concatenated data
        text_data = df.apply(
            lambda row: f"Date: {row['Date']}, Open: {row['Open']}, High: {row['High']}, Low: {row['Low']}, Close: {row['Close']}, Adj Close: {row['Adj Close']}, Volume: {row['Volume']}",
            axis=1
        )

        # Generate embeddings for the stock data
        embeddings = embedding_model.encode(text_data.tolist(), convert_to_numpy=True)
        
        # Prepare data for Pinecone upsert
        vectors = [(f"{symbol}_{i}", embeddings[i].tolist(), {'symbol': symbol}) for i in range(len(embeddings))]
        
        # Upsert data into Pinecone index
        try:
            response = index.upsert(vectors)
            logger.info(f"Successfully upserted {len(vectors)} vectors for {symbol}.")
            logger.info(f"Pinecone response: {response}")
        except Exception as e:
            logger.error(f"Error upserting data into Pinecone for {symbol}: {e}")
        
        # Save the processed DataFrame to a new CSV file in the output folder
        processed_file_path = os.path.join(output_folder, f'processed_{symbol}.csv')
        df.to_csv(processed_file_path, index=False)
        logger.info(f"Processed data saved to {processed_file_path}")
    
    except Exception as e:
        logger.error(f"Error processing data for {symbol}: {e}")

In [10]:
# Loop through all symbols and process each
for symbol in symbols:
    fetch_and_save_stock_data(symbol)

[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for NVDA saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2\NVDA.csv


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

C:\Users\Pham Ty\AppData\Roaming\Python\Python312\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
ERROR:__main__:Error upserting data into Pinecone for NVDA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 23 Sep 2024 15:07:20 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '3091', 'x-pinecone-request-id': '8024494975870077241', 'x-envoy-upstream-service-time': '3', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4915154 bytes, the limit is: 4194304 bytes","details":[]}

INFO:__main__:Processed data saved to C:\Users\Pham Ty\Desktop\Thesis-Pred

Batches:   0%|          | 0/79 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for INTC: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 23 Sep 2024 15:07:26 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1633', 'x-pinecone-request-id': '3211537696267647456', 'x-envoy-upstream-service-time': '3', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4915154 bytes, the limit is: 4194304 bytes","details":[]}

INFO:__main__:Processed data saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\processed_INTC.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for PLTR saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2\PLTR.csv


...

ERROR:__main__:Error upserting data into Pinecone for CMG: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 23 Sep 2024 15:11:43 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1582', 'x-pinecone-request-id': '6272663966791698130', 'x-envoy-upstream-service-time': '2', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4910122 bytes, the limit is: 4194304 bytes","details":[]}

INFO:__main__:Processed data saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\processed_CMG.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for BCS saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2\BCS.csv


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for BCS: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 23 Sep 2024 15:11:49 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1708', 'x-pinecone-request-id': '8482135694404558734', 'x-envoy-upstream-service-time': '3', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4910122 bytes, the limit is: 4194304 bytes","details":[]}

INFO:__main__:Processed data saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\processed_BCS.csv
[*********************100%***********************]  1 of 1 completed
INFO:__main__:Data for UAA saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2\UAA.csv


Batches:   0%|          | 0/79 [00:00<?, ?it/s]

ERROR:__main__:Error upserting data into Pinecone for UAA: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Mon, 23 Sep 2024 15:11:55 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '1326', 'x-pinecone-request-id': '5318532030436758107', 'x-envoy-upstream-service-time': '12', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4910122 bytes, the limit is: 4194304 bytes","details":[]}

INFO:__main__:Processed data saved to C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\processed_UAA.csv


In [22]:
csv_files = ['NVDA.csv', 'INTC.csv', 'PLTR.csv', 'TSLA.csv', 'AAPL.csv', 'BBD.csv', 'T.csv', 'SOFI.csv',
             'WBD.csv', 'SNAP.csv', 'NIO.csv', 'BTG.csv', 'F.csv', 'AAL.csv', 'NOK.csv', 'BAC.csv', 
             'CCL.csv', 'ORCL.csv', 'AMD.csv', 'PFE.csv', 'KGC.csv', 'MARA.csv', 'SLB.csv', 'NU.csv', 
             'MPW.csv', 'MU.csv', 'LCID.csv', 'NCLH.csv', 'RIG.csv', 'AMZN.csv', 'ABEV.csv', 'U.csv', 
             'LUMN.csv', 'AGNC.csv', 'VZ.csv', 'WBA.csv', 'WFC.csv', 'RIVN.csv', 'UPST.csv', 'GRAB.csv', 
             'CSCO.csv', 'VALE.csv', 'AVGO.csv', 'PBR.csv', 'GOOGL.csv', 'SMMT.csv', 'GOLD.csv', 
             'CMG.csv', 'BCS.csv', 'UAA.csv']

# Kiểm tra các tệp có trong thư mục
csv_files_in_folder = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Lấy các mã từ tên các file CSV
csv_symbols_in_folder = [f.split('.')[0] for f in csv_files_in_folder]

# Kiểm tra các mã cổ phiếu nào thiếu
missing_symbols = [symbol for symbol in symbols if symbol not in csv_symbols_in_folder]

# In ra kết quả
if missing_symbols:
    print(f"Các mã cổ phiếu bị thiếu: {missing_symbols}")
else:
    print("Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.")

Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.


In [8]:
# Import necessary libraries
import os
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from joblib import Parallel, delayed
import multiprocessing
import math
import warnings
warnings.filterwarnings('ignore')

In [9]:
# Function to load stock data from CSV
def load_stock_data(symbol):
    file_path = os.path.join(folder_path, f'{symbol}.csv')
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df = df[['Date', 'Close']]
    df = df.sort_values('Date')  # Sắp xếp theo ngày tăng dần
    return df

In [10]:
# Function to prepare data for LSTM model
def prepare_data(df, time_step=60):
    data = df['Close'].values
    data = data.reshape(-1, 1)

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    # Prepare sequences
    x_data, y_data = [], []
    for i in range(time_step, len(scaled_data)):
        x_data.append(scaled_data[i - time_step:i, 0])
        y_data.append(scaled_data[i, 0])

    # Convert to numpy arrays
    x_data, y_data = np.array(x_data), np.array(y_data)

    # Reshape for LSTM [samples, time_steps, features]
    x_data = np.reshape(x_data, (x_data.shape[0], x_data.shape[1], 1))

    return x_data, y_data, scaler

In [11]:
# Function to build LSTM model
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(input_shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=25))
    model.add(Dense(units=1))  # Output layer

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [19]:
def process_symbol(symbol):
    print(f"Processing {symbol}...")
    df = load_stock_data(symbol)
    x_data, y_data, scaler = prepare_data(df)

    # Chia dữ liệu thành 8 năm train và 2 năm test
    train_size = int(len(x_data) * (8 / 10))  # 8 năm trong tổng số 10 năm
    x_train, x_test = x_data[:train_size], x_data[train_size:]
    y_train, y_test = y_data[:train_size], y_data[train_size:]

    # Inverse transform y_train và y_test để có giá trị gốc
    y_train_original = scaler.inverse_transform(y_train.reshape(-1, 1)).flatten().tolist()
    y_test_original = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten().tolist()

    # Xây dựng và huấn luyện mô hình
    model = build_lstm_model(x_train.shape)
    model.fit(x_train, y_train, batch_size=64, epochs=10, verbose=0)

    # Dự đoán giá trên tập test
    predicted_prices = model.predict(x_test)
    predicted_prices = scaler.inverse_transform(predicted_prices).flatten().tolist()

    # Tính toán các chỉ số đánh giá
    mse = mean_squared_error(y_test_original, predicted_prices)
    rmse = math.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test_original, predicted_prices)

    # Dự báo giá cho 1 năm tiếp theo (nếu cần)
    future_prices = []
    last_sequence = x_test[-1]
    for _ in range(365):
        next_price = model.predict(last_sequence.reshape(1, -1, 1))
        future_price = scaler.inverse_transform(next_price)
        future_prices.append(future_price[0, 0])
        last_sequence = np.append(last_sequence[1:], next_price, axis=0)

    # Trả về kết quả, bao gồm cả Train_Prices và Test_Prices
    return {
        'Symbol': symbol,
        'RMSE': rmse,
        'MSE': mse,
        'MAPE': mape,
        'Predicted_Prices': predicted_prices,
        'Actual_Prices': y_test_original,
        'Future_Price_Predictions': future_prices,
        'Train_Prices': y_train_original
    }

# Main processing block
if __name__ == '__main__':
    # Create a list to store forecast results
    forecast_results = []

    # Parallel processing for all symbols
    num_cores = multiprocessing.cpu_count()
    results = Parallel(n_jobs=num_cores)(
        delayed(process_symbol)(symbol) for symbol in symbols
    )

    # Add the results to forecast_results list
    forecast_results.extend(results)

    # Create the summary DataFrame
    summary_df = pd.DataFrame(forecast_results)

    # Convert lists to strings for saving
    summary_df['Predicted_Prices'] = summary_df['Predicted_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Actual_Prices'] = summary_df['Actual_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Future_Price_Predictions'] = summary_df['Future_Price_Predictions'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Train_Prices'] = summary_df['Train_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )

    # Save the summary DataFrame to CSV
    summary_file = os.path.join(output_folder, 'forecast_summary.csv')
    summary_df.to_csv(summary_file, index=False)

    print(f"Tất cả kết quả đã được lưu trong {summary_file}")

    # Print the summary DataFrame
    print("\nBảng tổng hợp kết quả dự báo:")
    print(summary_df)

Tất cả kết quả đã được lưu trong C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\forecast_summary.csv

Bảng tổng hợp kết quả dự báo:
   Symbol       RMSE         MSE      MAPE  \
0    NVDA   7.265612   52.789114  0.072978   
1    INTC   2.037902    4.153046  0.046167   
2    PLTR   2.022444    4.090278  0.052500   
3    TSLA  15.526008  241.056921  0.060866   
4    AAPL   8.936813   79.866622  0.040319   
5     BBD   0.148475    0.022045  0.037604   
6       T   0.635786    0.404223  0.027930   
7    SOFI   0.495193    0.245216  0.051159   
8     WBD   0.860466    0.740403  0.062352   
9    SNAP   1.245678    1.551714  0.075383   
10    NIO   1.036498    1.074328  0.125148   
11    BTG   0.186767    0.034882  0.044442   
12      F   0.652790    0.426134  0.039994   
13    AAL   0.732899    0.537140  0.041010   
14    NOK   0.142649    0.020349  0.026693   
15    BAC   1.261604    1.591644  0.029096   
16    CCL   1.041948    1.085655  0.062258   
17   ORCL   5.002069   25.0

In [12]:
import os
import pandas as pd
import numpy as np
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
from datetime import timedelta

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Paths to data
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'  # Folder with stock CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'  # Output folder

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

# Load forecast_summary.csv
forecast_summary_file = os.path.join(output_folder, 'forecast_summary.csv')
if os.path.exists(forecast_summary_file):
    forecast_summary_df = pd.read_csv(forecast_summary_file)
    # Parse string lists into actual lists
    for col in ['Predicted_Prices', 'Actual_Prices', 'Future_Price_Predictions', 'Train_Prices']:
        if col in forecast_summary_df.columns:
            forecast_summary_df[col] = forecast_summary_df[col].apply(
                lambda x: [float(item.strip()) for item in x.split(',')] if pd.notnull(x) else []
            )
    logging.info("Loaded 'forecast_summary.csv' successfully.")
else:
    forecast_summary_df = pd.DataFrame()
    logging.warning("Could not find 'forecast_summary.csv'. Evaluation metrics and forecasts will not be displayed.")

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define app layout
app.layout = html.Div([
    html.H1('Stock Price Predictions Dashboard'),
    html.Div([
        html.Label('Select Stock Symbol'),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': s, 'value': s} for s in symbols],
            value='NVDA'  # Default stock
        )
    ], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Select Date Range'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed='2014-09-18',  # Adjusted min date for 10-year period
            max_date_allowed='2024-09-18',  # Adjusted max date for 10-year period
            start_date='2014-09-18',        # Default start date
            end_date='2024-09-18'           # Default end date
        )
    ], style={'display': 'inline-block', 'marginLeft': '50px'}),
    html.Div([
        html.Label('Show 1-Year Forecast'),
        dcc.Checklist(
            id='forecast-checkbox',
            options=[{'label': 'Include 1-Year Forecast', 'value': 'show_forecast'}],
            value=[],
            inline=True
        )
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='metrics-output', style={'marginTop': '20px'})
])

# Define callback to update graph and evaluation metrics
@app.callback(
    [Output('price-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('forecast-checkbox', 'value')]
)
def update_graph(selected_stock, start_date, end_date, forecast_option):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"No forecast data found for {selected_stock}."

    # Parse 'Actual_Prices', 'Predicted_Prices', 'Future_Price_Predictions', 'Train_Prices'
    actual_prices = forecast_row.iloc[0]['Actual_Prices']
    predicted_prices = forecast_row.iloc[0]['Predicted_Prices']
    future_prices = forecast_row.iloc[0]['Future_Price_Predictions']
    train_prices = forecast_row.iloc[0]['Train_Prices']

    # Load the stock data to get dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Stock data file not found for {selected_stock}."

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Incorrect data format in stock data file for {selected_stock}."

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    dates = df_stock['Date']
    data_length = len(df_stock)

    time_step = 60  # As used in your model
    total_samples = data_length - time_step

    # Calculate train_size and test_size based on 10-year data
    # Assuming 8 years for training and 2 years for testing
    # Calculate the number of samples per year
    # For simplicity, assume 252 trading days per year
    samples_per_year = 252
    train_size = 8 * samples_per_year
    test_size = 2 * samples_per_year

    # Adjust train_size and test_size if total_samples is less
    if train_size + test_size > total_samples:
        train_size = int(total_samples * 0.8)
        test_size = total_samples - train_size
        logging.warning(f"Adjusted train_size to {train_size} and test_size to {test_size} due to insufficient data.")

    # Ensure that the length of 'Actual_Prices' matches 'test_size'
    if len(actual_prices) != test_size or len(predicted_prices) != test_size:
        logging.error(f"Length mismatch between actual/predicted prices and test set for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Length mismatch in data for {selected_stock}."

    # Get indices for train and test sets in original data
    train_indices_in_y_data = range(0, train_size)
    test_indices_in_y_data = range(train_size, train_size + test_size)
    indices_train = [i + time_step for i in train_indices_in_y_data]
    indices_test = [i + time_step for i in test_indices_in_y_data]

    # Ensure indices do not exceed the length of dates
    indices_train = [i for i in indices_train if i < len(dates)]
    indices_test = [i for i in indices_test if i < len(dates)]

    # Dates for train and test sets
    dates_train = dates.iloc[indices_train].reset_index(drop=True)
    dates_test = dates.iloc[indices_test].reset_index(drop=True)

    # Create DataFrame for train set
    df_train = pd.DataFrame({
        'Date': dates_train,
        'Train_Price': train_prices
    })

    # Create DataFrame for test set
    df_test = pd.DataFrame({
        'Date': dates_test,
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })

    # Filter df_train and df_test based on 'start_date' and 'end_date'
    df_train_filtered = df_train[(df_train['Date'] >= start_date) & (df_train['Date'] <= end_date)]
    df_test_filtered = df_test[(df_test['Date'] >= start_date) & (df_test['Date'] <= end_date)]

    # Initialize data list for plotting
    data = []

    # Add Train Prices if available in the selected date range
    if not df_train_filtered.empty:
        data.append(
            go.Scatter(
                x=df_train_filtered['Date'],
                y=df_train_filtered['Train_Price'],
                mode='lines',
                name='Train Price',
                line=dict(color='red')
            )
        )

    # Add Actual Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Actual_Price'],
                mode='lines',
                name='Actual Test Price',
                line=dict(color='green')
            )
        )

    # Add Predicted Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Predicted_Price'],
                mode='lines',
                name='Predicted Test Price',
                line=dict(color='blue')
            )
        )

    # Add future price predictions if selected
    if 'show_forecast' in forecast_option:
        # Generate future dates
        last_date = dates.iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')

        if len(future_prices) != len(future_dates):
            logging.error(f"Length mismatch between future prices and future dates for {selected_stock}.")
        else:
            data.append(
                go.Scatter(
                    x=future_dates,
                    y=future_prices,
                    mode='lines',
                    name='1-Year Forecast',
                    line=dict(dash='dash', color='orange')
                )
            )

    # Create figure
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f'{selected_stock} Price Prediction (2014-09-18 to 2024-09-18)',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'},
            hovermode='closest'
        )
    }

    # Get RMSE, MSE, and MAPE
    rmse = forecast_row.iloc[0]['RMSE']
    mse = forecast_row.iloc[0]['MSE']
    mape = forecast_row.iloc[0]['MAPE']

    metrics_output = [
        html.P(f'Root Mean Squared Error (RMSE): {rmse:.4f}'),
        html.P(f'Mean Squared Error (MSE): {mse:.4f}'),
        html.P(f'Mean Absolute Percentage Error (MAPE): {mape:.4%}')
    ]

    return figure, metrics_output

# Run Dash app
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)

INFO:root:Loaded 'forecast_summary.csv' successfully.


In [3]:
import os
import pandas as pd
import numpy as np
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
from datetime import timedelta

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Paths to data
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'  # Folder with stock CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'  # Output folder

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

# Load forecast_summary.csv
forecast_summary_file = os.path.join(output_folder, 'forecast_summary.csv')
if os.path.exists(forecast_summary_file):
    forecast_summary_df = pd.read_csv(forecast_summary_file)
    # Parse string lists into actual lists
    for col in ['Predicted_Prices', 'Actual_Prices', 'Future_Price_Predictions', 'Train_Prices']:
        if col in forecast_summary_df.columns:
            forecast_summary_df[col] = forecast_summary_df[col].apply(
                lambda x: [float(item.strip()) for item in x.split(',')] if pd.notnull(x) else []
            )
    logging.info("Loaded 'forecast_summary.csv' successfully.")
else:
    forecast_summary_df = pd.DataFrame()
    logging.warning("Could not find 'forecast_summary.csv'. Evaluation metrics and forecasts will not be displayed.")

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define app layout
app.layout = html.Div([
    html.H1('Stock Price Predictions Dashboard'),
    html.Div([
        html.Label('Select Stock Symbol'),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': s, 'value': s} for s in symbols],
            value='NVDA'  # Default stock
        )
    ], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Select Date Range'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed='2014-09-18',  # Adjusted min date for 10-year period
            max_date_allowed='2024-09-18',  # Adjusted max date for 10-year period
            start_date='2014-09-18',        # Default start date
            end_date='2024-09-18'           # Default end date
        )
    ], style={'display': 'inline-block', 'marginLeft': '50px'}),
    html.Div([
        html.Label('Show 1-Year Forecast'),
        dcc.Checklist(
            id='forecast-checkbox',
            options=[{'label': 'Include 1-Year Forecast', 'value': 'show_forecast'}],
            value=[],
            inline=True
        )
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='metrics-output', style={'marginTop': '20px'})
])

# Define callback to update graph and evaluation metrics
@app.callback(
    [Output('price-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('forecast-checkbox', 'value')]
)
def update_graph(selected_stock, start_date, end_date, forecast_option):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"No forecast data found for {selected_stock}."

    # Parse 'Actual_Prices', 'Predicted_Prices', 'Future_Price_Predictions', 'Train_Prices'
    actual_prices = forecast_row.iloc[0]['Actual_Prices']
    predicted_prices = forecast_row.iloc[0]['Predicted_Prices']
    future_prices = forecast_row.iloc[0]['Future_Price_Predictions']
    train_prices = forecast_row.iloc[0]['Train_Prices']

    # Load the stock data to get dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Stock data file not found for {selected_stock}."

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Incorrect data format in stock data file for {selected_stock}."

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    dates = df_stock['Date']
    data_length = len(df_stock)

    time_step = 60  # As used in your model
    total_samples = data_length - time_step

    # Calculate train_size and test_size based on 10-year data
    # Assuming 8 years for training and 2 years for testing
    # Calculate the number of samples per year
    # For simplicity, assume 252 trading days per year
    samples_per_year = 252
    train_size = 8 * samples_per_year
    test_size = 2 * samples_per_year

    # Adjust train_size and test_size if total_samples is less
    if train_size + test_size > total_samples:
        train_size = int(total_samples * 0.8)
        test_size = total_samples - train_size
        logging.warning(f"Adjusted train_size to {train_size} and test_size to {test_size} due to insufficient data.")

    # Ensure that the length of 'Actual_Prices' matches 'test_size'
    if len(actual_prices) != test_size or len(predicted_prices) != test_size:
        logging.error(f"Length mismatch between actual/predicted prices and test set for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Length mismatch in data for {selected_stock}."

    # Get indices for train and test sets in original data
    train_indices_in_y_data = range(0, train_size)
    test_indices_in_y_data = range(train_size, train_size + test_size)
    indices_train = [i + time_step for i in train_indices_in_y_data]
    indices_test = [i + time_step for i in test_indices_in_y_data]

    # Ensure indices do not exceed the length of dates
    indices_train = [i for i in indices_train if i < len(dates)]
    indices_test = [i for i in indices_test if i < len(dates)]

    # Dates for train and test sets
    dates_train = dates.iloc[indices_train].reset_index(drop=True)
    dates_test = dates.iloc[indices_test].reset_index(drop=True)

    # Create DataFrame for train set
    df_train = pd.DataFrame({
        'Date': dates_train,
        'Train_Price': train_prices
    })

    # Create DataFrame for test set
    df_test = pd.DataFrame({
        'Date': dates_test,
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })

    # Determine the split date (end of train)
    if not df_train.empty:
        split_date = df_train['Date'].max()
    else:
        split_date = None
        logging.warning(f"Train data is empty for {selected_stock}.")

    # Filter df_train and df_test based on 'start_date' and 'end_date'
    df_train_filtered = df_train[(df_train['Date'] >= start_date) & (df_train['Date'] <= end_date)]
    df_test_filtered = df_test[(df_test['Date'] >= start_date) & (df_test['Date'] <= end_date)]

    # Initialize data list for plotting
    data = []

    # Add Train Prices if available in the selected date range
    if not df_train_filtered.empty:
        data.append(
            go.Scatter(
                x=df_train_filtered['Date'],
                y=df_train_filtered['Train_Price'],
                mode='lines',
                name='Train Price',
                line=dict(color='red')
            )
        )

    # Add Actual Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Actual_Price'],
                mode='lines',
                name='Actual Test Price',
                line=dict(color='green')
            )
        )

    # Add Predicted Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Predicted_Price'],
                mode='lines',
                name='Predicted Test Price',
                line=dict(color='blue')
            )
        )

    # Add vertical line to indicate the split between train and test
    if split_date:
        data.append(
            go.Scatter(
                x=[split_date, split_date],
                y=[df_stock['Close'].min(), df_stock['Close'].max()],
                mode='lines',
                name='Train-Test Split',
                line=dict(color='black', dash='dash')
            )
        )

    # Add future price predictions if selected
    if 'show_forecast' in forecast_option:
        # Generate future dates
        last_date = dates.iloc[-1]
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')

        if len(future_prices) != len(future_dates):
            logging.error(f"Length mismatch between future prices and future dates for {selected_stock}.")
        else:
            data.append(
                go.Scatter(
                    x=future_dates,
                    y=future_prices,
                    mode='lines',
                    name='1-Year Forecast',
                    line=dict(dash='dash', color='orange')
                )
            )

    # Create figure
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f'{selected_stock} Price Prediction (2014-09-18 to 2024-09-18)',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'},
            hovermode='closest'
        )
    }

    # Get RMSE, MSE, and MAPE
    rmse = forecast_row.iloc[0]['RMSE']
    mse = forecast_row.iloc[0]['MSE']
    mape = forecast_row.iloc[0]['MAPE']

    metrics_output = [
        html.P(f'Root Mean Squared Error (RMSE): {rmse:.4f}'),
        html.P(f'Mean Squared Error (MSE): {mse:.4f}'),
        html.P(f'Mean Absolute Percentage Error (MAPE): {mape:.4%}')
    ]

    return figure, metrics_output

# Run Dash app
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)

2024-10-21 20:52:38,783 INFO:Loaded 'forecast_summary.csv' successfully.


2024-10-21 20:52:39,209 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 20:52:41,786 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 20:52:45,226 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 20:52:48,580 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 20:52:50,206 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 20:52:51,982 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.
2024-10-21 20:52:54,784 WARNING:Adjusted train_size to 1964 and test_size to 492 due to insufficient data.


PROPHET MODEL

In [3]:
pip install prophet

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/13.3 MB 2.0 MB/s eta 0:00:07
    --------------------------------------- 0.2/13.3 MB 2.8 MB/s eta 0:00:05
   -- ------------------------------------- 0.7/13.3 MB 5.7 MB/s eta 0:00:03
   ----- ---------------------------------- 1.7/13.3 MB 10.1 MB/s eta 0:00:02
   ------------ --------------------------- 4.0/13.3 MB 18.4 MB/s eta 0:00:01
   ---------------- ----------------------- 5.5/13.3 MB 20.9 MB/s eta 0:00:01
   ----------------- ---------------------- 5.8/13.3 MB 18.5 MB/s eta 0:00:01
   --------------------- ------------------ 7.1/13.3 MB 19.6 MB/s eta 0:00:01
   ----------------------- ---------------- 7.7/13.3 MB 19.8 MB/s eta 0:00:01
   -------------------------- ------------- 8.8/13.3 MB 20.2 MB/s eta 0:00:01
   --------------------------- ------------ 9.2/13.3 MB 19.6 MB/s eta 0:00:

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [4]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import logging
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Pinecone
pc = Pinecone(api_key="09bb980b-6cef-48c3-9aa5-63f3cbc9885e")
index = pc.Index("financial-data-index")

# Initialize SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
logger.info("SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.")

# Folder containing the CSV files
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'

# Folder to save the processed CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
    'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
    'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
    'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
    'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
    'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
    'CMG', 'BCS', 'UAA']

def fetch_and_save_stock_data(symbol, start_date='2014-09-18', end_date='2024-09-18'):
    """
    Fetch stock data from Yahoo Finance, save to CSV, and upsert data into Pinecone.
    """
    try:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        
        # Save data to CSV
        csv_file_path = os.path.join(folder_path, f'{symbol}.csv')
        stock_data.to_csv(csv_file_path)
        logger.info(f"Data for {symbol} saved to {csv_file_path}")
        
        # Load CSV data
        df = pd.read_csv(csv_file_path)
        
        # Select necessary columns and drop NaN values
        required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"{csv_file_path} is missing one or more required columns: {required_columns}")
            return
        
        df = df[required_columns].dropna()

        # Convert each row into a single string of concatenated data
        text_data = df.apply(
            lambda row: f"Date: {row['Date']}, Open: {row['Open']}, High: {row['High']}, Low: {row['Low']}, Close: {row['Close']}, Adj Close: {row['Adj Close']}, Volume: {row['Volume']}",
            axis=1
        )

        # Generate embeddings for the stock data
        embeddings = embedding_model.encode(text_data.tolist(), convert_to_numpy=True)
        
        # Prepare data for Pinecone upsert
        vectors = [(f"{symbol}_{i}", embeddings[i].tolist(), {'symbol': symbol}) for i in range(len(embeddings))]
        
        # Upsert data into Pinecone index
        try:
            response = index.upsert(vectors)
            logger.info(f"Successfully upserted {len(vectors)} vectors for {symbol}.")
            logger.info(f"Pinecone response: {response}")
        except Exception as e:
            logger.error(f"Error upserting data into Pinecone for {symbol}: {e}")
        
        # Save the processed DataFrame to a new CSV file in the output folder
        processed_file_path = os.path.join(output_folder, f'processed_{symbol}.csv')
        df.to_csv(processed_file_path, index=False)
        logger.info(f"Processed data saved to {processed_file_path}")
    
    except Exception as e:
        logger.error(f"Error processing data for {symbol}: {e}")

# List of expected CSV files
csv_files = [f'{symbol}.csv' for symbol in symbols]

# Check existing CSV files in the folder
csv_files_in_folder = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Extract symbols from existing CSV filenames
csv_symbols_in_folder = [f.split('.')[0] for f in csv_files_in_folder]

# Identify missing symbols
missing_symbols = [symbol for symbol in symbols if symbol not in csv_symbols_in_folder]

# Print the result
if missing_symbols:
    print(f"Các mã cổ phiếu bị thiếu: {missing_symbols}")
else:
    print("Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.")

# Import necessary libraries for forecasting
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from prophet import Prophet  # Changed from fbprophet to prophet
from joblib import Parallel, delayed
import multiprocessing
import math

# Function to load stock data from CSV
def load_stock_data(symbol):
    file_path = os.path.join(folder_path, f'{symbol}.csv')
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df = df[['Date', 'Close']]
    df = df.sort_values('Date')  # Sort by date ascending
    return df

# Function to prepare data for Prophet model
def prepare_data_prophet(df):
    # Rename columns to 'ds' and 'y' as required by Prophet
    df_prophet = df.rename(columns={'Date': 'ds', 'Close': 'y'})
    return df_prophet

# Function to process each symbol using Prophet
def process_symbol(symbol):
    print(f"Processing {symbol}...")
    try:
        df = load_stock_data(symbol)
        df_prophet = prepare_data_prophet(df)
        
        # Split data into train (first 8 years) and test (last 2 years)
        train_end_date = df_prophet['ds'].min() + pd.DateOffset(years=8)
        train_df = df_prophet[df_prophet['ds'] <= train_end_date]
        test_df = df_prophet[df_prophet['ds'] > train_end_date]
        
        # Initialize and fit Prophet model
        model = Prophet(daily_seasonality=False, yearly_seasonality=True, weekly_seasonality=True)
        model.fit(train_df)
        
        # Create dataframe for test period
        future = test_df[['ds']].copy()
        forecast = model.predict(future)
        
        # Extract the predicted 'yhat'
        predicted = forecast['yhat'].tolist()
        actual = test_df['y'].tolist()
        
        # Calculate evaluation metrics
        mse = mean_squared_error(actual, predicted)
        rmse = math.sqrt(mse)
        mape = mean_absolute_percentage_error(actual, predicted)
        
        # Forecast future prices for 1 year ahead
        future_dates = model.make_future_dataframe(periods=365)
        future_forecast = model.predict(future_dates)
        future_prices = future_forecast['yhat'][-365:].tolist()
        
        # Train prices (actual training data)
        train_prices = train_df['y'].tolist()
        
        return {
            'Symbol': symbol,
            'RMSE': rmse,
            'MSE': mse,
            'MAPE': mape,
            'Predicted_Prices': predicted,
            'Actual_Prices': actual,
            'Future_Price_Predictions': future_prices,
            'Train_Prices': train_prices
        }
    
    except Exception as e:
        logger.error(f"Error processing symbol {symbol} with Prophet: {e}")
        return {
            'Symbol': symbol,
            'RMSE': None,
            'MSE': None,
            'MAPE': None,
            'Predicted_Prices': [],
            'Actual_Prices': [],
            'Future_Price_Predictions': [],
            'Train_Prices': []
        }

# Main processing block
if __name__ == '__main__':
    # Create a list to store forecast results
    forecast_results = []

    # Parallel processing for all symbols
    num_cores = multiprocessing.cpu_count()
    results = Parallel(n_jobs=num_cores)(
        delayed(process_symbol)(symbol) for symbol in symbols
    )

    # Add the results to forecast_results list
    forecast_results.extend(results)

    # Create the summary DataFrame
    summary_df = pd.DataFrame(forecast_results)

    # Convert lists to strings for saving
    summary_df['Predicted_Prices'] = summary_df['Predicted_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Actual_Prices'] = summary_df['Actual_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Future_Price_Predictions'] = summary_df['Future_Price_Predictions'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Train_Prices'] = summary_df['Train_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )

    # Save the summary DataFrame to CSV with the specified filename
    summary_file = os.path.join(output_folder, 'forecast_summary_Prophet.csv')  # Updated filename
    summary_df.to_csv(summary_file, index=False)

    print(f"Tất cả kết quả đã được lưu trong {summary_file}")

    # Print the summary DataFrame
    print("\nBảng tổng hợp kết quả dự báo:")
    print(summary_df)

INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['C:\\Users\\Pham Ty\\AppData\\Roaming\\Python\\Python312\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.


Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.
Tất cả kết quả đã được lưu trong C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\forecast_summary_Prophet.csv

Bảng tổng hợp kết quả dự báo:
   Symbol        RMSE           MSE      MAPE  \
0    NVDA   43.223425   1868.264492  0.495865   
1    INTC   14.109645    199.082075  0.353835   
2    PLTR         NaN           NaN       NaN   
3    TSLA  167.091559  27919.589035  0.835795   
4    AAPL   21.253842    451.725815  0.110963   
5     BBD    0.600685      0.360822  0.165139   
6       T    3.083694      9.509168  0.139693   
7    SOFI         NaN           NaN       NaN   
8     WBD   18.259470    333.408238  1.670443   
9    SNAP         NaN           NaN       NaN   
10    NIO         NaN           NaN       NaN   
11    BTG    0.899514      0.809125  0.260761   
12      F    9.043384     81.782797  0.726426   
13    AAL    2.056905      4.230859  0.115432   
14    NOK    2.143053      4.592675  0.525973   
15    BAC    9.4267

In [2]:
import os
import pandas as pd
import numpy as np
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
from datetime import timedelta
import ast

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Paths to data
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'  # Folder with stock CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'  # Output folder

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

# Load forecast_summary_Prophet.csv
forecast_summary_file = os.path.join(output_folder, 'forecast_summary_Prophet.csv')
if os.path.exists(forecast_summary_file):
    forecast_summary_df = pd.read_csv(forecast_summary_file)
    # Parse string lists into actual lists
    for col in ['Predicted_Prices', 'Actual_Prices', 'Future_Price_Predictions', 'Train_Prices']:
        if col in forecast_summary_df.columns:
            forecast_summary_df[col] = forecast_summary_df[col].apply(
                lambda x: [float(item.strip()) for item in x.split(',')] if pd.notnull(x) else []
            )
    logging.info("Loaded 'forecast_summary_Prophet.csv' successfully.")
else:
    forecast_summary_df = pd.DataFrame()
    logging.warning("Could not find 'forecast_summary_Prophet.csv'. Evaluation metrics and forecasts will not be displayed.")

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define app layout
app.layout = html.Div([
    html.H1('Stock Price Predictions Dashboard (Prophet)'),
    html.Div([
        html.Label('Select Stock Symbol'),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': s, 'value': s} for s in symbols],
            value='NVDA'  # Default stock
        )
    ], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Select Date Range'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed='2014-09-18',  # Adjusted min date for 10-year period
            max_date_allowed='2025-12-31',  # Extended max date to accommodate future forecasts
            start_date='2014-09-18',        # Default start date
            end_date='2025-09-18'           # Extended default end date to include 1-year forecast
        )
    ], style={'display': 'inline-block', 'marginLeft': '50px'}),
    html.Div([
        html.Label('Show 1-Year Forecast'),
        dcc.Checklist(
            id='forecast-checkbox',
            options=[{'label': 'Include 1-Year Forecast', 'value': 'show_forecast'}],
            value=[],  # Default unchecked
            inline=True
        )
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='metrics-output', style={'marginTop': '20px'})
])

# Define callback to update graph and evaluation metrics
@app.callback(
    [Output('price-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('forecast-checkbox', 'value')]
)
def update_graph(selected_stock, start_date, end_date, forecast_option):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"No forecast data found for {selected_stock}."

    # Parse 'Actual_Prices', 'Predicted_Prices', 'Future_Price_Predictions', 'Train_Prices'
    actual_prices = forecast_row.iloc[0]['Actual_Prices']
    predicted_prices = forecast_row.iloc[0]['Predicted_Prices']
    future_prices = forecast_row.iloc[0]['Future_Price_Predictions']
    train_prices = forecast_row.iloc[0]['Train_Prices']

    # Load the stock data to get dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Stock data file not found for {selected_stock}."

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Incorrect data format in stock data file for {selected_stock}."

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    df_stock_sorted = df_stock.sort_values('Date')
    dates = df_stock_sorted['Date']
    data_length = len(df_stock_sorted)

    # Derive split_date based on 8-year training period
    split_date = df_stock_sorted['Date'].min() + pd.DateOffset(years=8)

    # Get dates for Train and Test based on split_date
    train_mask = df_stock_sorted['Date'] <= split_date
    test_mask = df_stock_sorted['Date'] > split_date

    df_train = df_stock_sorted[train_mask]
    df_test = df_stock_sorted[test_mask]

    # Ensure that 'Actual_Prices' and 'Predicted_Prices' correspond to the test set
    test_dates = df_test['Date'].reset_index(drop=True)
    actual_prices = actual_prices[:len(test_dates)]  # Trim if necessary
    predicted_prices = predicted_prices[:len(test_dates)]

    if len(actual_prices) != len(predicted_prices):
        logging.error(f"Length mismatch between actual and predicted prices for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Length mismatch in data for {selected_stock}."

    # Create DataFrame for Train set within selected date range
    df_train_filtered = df_train[(df_train['Date'] >= start_date) & (df_train['Date'] <= end_date)]

    # Create DataFrame for Test set within selected date range
    df_test_filtered = pd.DataFrame({
        'Date': test_dates[:len(actual_prices)],
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })
    df_test_filtered = df_test_filtered[
        (df_test_filtered['Date'] >= start_date) & (df_test_filtered['Date'] <= end_date)
    ]

    # Initialize data list for plotting
    data = []

    # Add Train Prices if available in the selected date range
    if not df_train_filtered.empty:
        data.append(
            go.Scatter(
                x=df_train_filtered['Date'],
                y=df_train_filtered['Close'],
                mode='lines',
                name='Train Price',
                line=dict(color='red')
            )
        )

    # Add Actual Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Actual_Price'],
                mode='lines',
                name='Actual Test Price',
                line=dict(color='green')
            )
        )

    # Add Predicted Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Predicted_Price'],
                mode='lines',
                name='Predicted Test Price',
                line=dict(color='blue')
            )
        )

    # Add vertical line to indicate the split between train and test
    if pd.notnull(split_date):
        data.append(
            go.Scatter(
                x=[split_date, split_date],
                y=[df_stock_sorted['Close'].min(), df_stock_sorted['Close'].max()],
                mode='lines',
                name='Train-Test Split',
                line=dict(color='black', dash='dash')
            )
        )

    # Add future price predictions if selected
    if 'show_forecast' in forecast_option:
        # Generate future dates
        last_date = df_stock_sorted['Date'].max()
        future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices), freq='D')

        if len(future_prices) != len(future_dates):
            logging.error(f"Length mismatch between future prices and future dates for {selected_stock}.")
        else:
            df_future = pd.DataFrame({
                'Date': future_dates,
                'Forecasted_Price': future_prices
            })

            # Filter future forecasts based on selected date range
            df_future_filtered = df_future[
                (df_future['Date'] >= start_date) & (df_future['Date'] <= end_date)
            ]

            if not df_future_filtered.empty:
                data.append(
                    go.Scatter(
                        x=df_future_filtered['Date'],
                        y=df_future_filtered['Forecasted_Price'],
                        mode='lines',
                        name='1-Year Forecast',
                        line=dict(dash='dash', color='orange')
                    )
                )
            else:
                logging.warning(f"No forecasted data within the selected date range for {selected_stock}.")

    # Create figure with dynamic title based on selected date range
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f'{selected_stock} Price Prediction ({start_date.date()} to {end_date.date()})',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'},
            hovermode='closest'
        )
    }

    # Get RMSE, MSE, and MAPE
    rmse = forecast_row.iloc[0]['RMSE']
    mse = forecast_row.iloc[0]['MSE']
    mape = forecast_row.iloc[0]['MAPE']

    # Format metrics output
    if pd.notnull(rmse) and pd.notnull(mse) and pd.notnull(mape):
        metrics_output = [
            html.P(f'Root Mean Squared Error (RMSE): {rmse:.4f}'),
            html.P(f'Mean Squared Error (MSE): {mse:.4f}'),
            html.P(f'Mean Absolute Percentage Error (MAPE): {mape:.2%}')
        ]
    else:
        metrics_output = [html.P("Evaluation metrics are not available.")]

    return figure, metrics_output

# Run Dash app
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)


2024-10-21 20:51:49,888 INFO:Loaded 'forecast_summary_Prophet.csv' successfully.


#XGBOOST MODEL

In [1]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 660.6 kB/s eta 0:03:10
   ---------------------------------------- 0.1/124.9 MB 991.0 kB/s eta 0:02:06
   ---------------------------------------- 0.3/124.9 MB 2.1 MB/s eta 0:00:59
   ---------------------------------------- 0.8/124.9 MB 4.4 MB/s eta 0:00:29
    --------------------------------------- 1.9/124.9 MB 8.1 MB/s eta 0:00:16
   - -------------------------------------- 4.2/124.9 MB 14.9 MB/s eta 0:00:09
   - -------------------------------------- 4.2/124.9 MB 14.9 MB/s eta 0:00:09
   - -------------------------------------- 4.2/124.9 MB 14.9 MB/s eta 0:00:09
   - -------------------------------------- 4.2/124.9 MB 14.9 MB/s eta 0:00:09
   - -------------------------------------- 4.2/124.9 MB 14.9 MB/s eta

In [6]:
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone
import logging
import yfinance as yf
import warnings
warnings.filterwarnings('ignore')

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Pinecone
pc = Pinecone(api_key="09bb980b-6cef-48c3-9aa5-63f3cbc9885e")
index = pc.Index("financial-data-index")

# Initialize SentenceTransformer model
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
logger.info("SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.")

# Folder containing the CSV files
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'

# Folder to save the processed CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'

# Create output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

def fetch_and_save_stock_data(symbol, start_date='2014-09-18', end_date='2024-09-18'):
    """
    Fetch stock data from Yahoo Finance, save to CSV, and upsert data into Pinecone.
    """
    try:
        # Fetch data from Yahoo Finance
        stock_data = yf.download(symbol, start=start_date, end=end_date)
        
        # Save data to CSV
        csv_file_path = os.path.join(folder_path, f'{symbol}.csv')
        stock_data.to_csv(csv_file_path)
        logger.info(f"Data for {symbol} saved to {csv_file_path}")
        
        # Load CSV data
        df = pd.read_csv(csv_file_path)
        
        # Select necessary columns and drop NaN values
        required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        if not all(col in df.columns for col in required_columns):
            logger.error(f"{csv_file_path} is missing one or more required columns: {required_columns}")
            return
        
        df = df[required_columns].dropna()

        # Convert each row into a single string of concatenated data
        text_data = df.apply(
            lambda row: f"Date: {row['Date']}, Open: {row['Open']}, High: {row['High']}, Low: {row['Low']}, Close: {row['Close']}, Adj Close: {row['Adj Close']}, Volume: {row['Volume']}",
            axis=1
        )

        # Generate embeddings for the stock data
        embeddings = embedding_model.encode(text_data.tolist(), convert_to_numpy=True)
        
        # Prepare data for Pinecone upsert
        vectors = [(f"{symbol}_{i}", embeddings[i].tolist(), {'symbol': symbol}) for i in range(len(embeddings))]
        
        # Upsert data into Pinecone index
        try:
            response = index.upsert(vectors)
            logger.info(f"Successfully upserted {len(vectors)} vectors for {symbol}.")
            logger.info(f"Pinecone response: {response}")
        except Exception as e:
            logger.error(f"Error upserting data into Pinecone for {symbol}: {e}")
        
        # Save the processed DataFrame to a new CSV file in the output folder
        processed_file_path = os.path.join(output_folder, f'processed_{symbol}.csv')
        df.to_csv(processed_file_path, index=False)
        logger.info(f"Processed data saved to {processed_file_path}")
    
    except Exception as e:
        logger.error(f"Error processing data for {symbol}: {e}")

# List of expected CSV files
csv_files = [f'{symbol}.csv' for symbol in symbols]

# Check existing CSV files in the folder
csv_files_in_folder = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Extract symbols from existing CSV filenames
csv_symbols_in_folder = [f.split('.')[0] for f in csv_files_in_folder]

# Identify missing symbols
missing_symbols = [symbol for symbol in symbols if symbol not in csv_symbols_in_folder]

# Print the result
if missing_symbols:
    print(f"Các mã cổ phiếu bị thiếu: {missing_symbols}")
else:
    print("Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.")

# Import necessary libraries for forecasting
import pandas as pd
import numpy as np
from datetime import timedelta
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from joblib import Parallel, delayed
import multiprocessing
import math
import xgboost as xgb  # Import XGBoost

# Function to load stock data from CSV
def load_stock_data(symbol):
    file_path = os.path.join(folder_path, f'{symbol}.csv')
    df = pd.read_csv(file_path, parse_dates=['Date'])
    df = df[['Date', 'Close']]
    df = df.sort_values('Date')  # Sort by date ascending
    return df

# Function to create lag features
def create_lag_features(df, lag=60):
    """
    Create lag features for time series forecasting.
    """
    for i in range(1, lag + 1):
        df[f'lag_{i}'] = df['Close'].shift(i)
    df = df.dropna().reset_index(drop=True)
    return df

# Function to prepare data for XGBoost model
def prepare_data_xgboost(df, lag=60):
    """
    Prepare data with lag features for XGBoost.
    """
    df_lag = create_lag_features(df, lag=lag)
    X = df_lag[[f'lag_{i}' for i in range(1, lag + 1)]].values
    y = df_lag['Close'].values
    return X, y

# Function to forecast future prices using XGBoost
def forecast_future_prices(model, last_features, periods=252):  # Changed periods to 252 for business days
    """
    Forecast future prices using the trained XGBoost model.
    """
    forecast = []
    features = last_features.copy()
    for _ in range(periods):
        pred = model.predict(features.reshape(1, -1))[0]
        forecast.append(pred)
        # Update features: remove the oldest lag and append the new prediction
        features = np.roll(features, -1)
        features[-1] = pred
    return forecast

# Function to process each symbol using XGBoost
def process_symbol_xgboost(symbol, lag=60):
    print(f"Processing {symbol} with XGBoost...")
    try:
        df = load_stock_data(symbol)
        
        # Prepare data with lag features
        X, y = prepare_data_xgboost(df, lag=lag)
        
        # Split data into train (first 80%) and test (last 20%)
        split_index = int(len(X) * 0.8)
        X_train, X_test = X[:split_index], X[split_index:]
        y_train, y_test = y[:split_index], y[split_index:]
        
        # Initialize and train XGBoost model
        model = xgb.XGBRegressor(
            objective='reg:squarederror',
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            random_state=42
        )
        model.fit(X_train, y_train)
        
        # Predict on test set
        y_pred = model.predict(X_test)
        
        # Calculate evaluation metrics
        mse = mean_squared_error(y_test, y_pred)
        rmse = math.sqrt(mse)
        mape = mean_absolute_percentage_error(y_test, y_pred)
        
        # Forecast future prices for 1 year ahead (252 business days)
        last_features = X[-1]  # Last available features from the training data
        future_prices = forecast_future_prices(model, last_features, periods=252)  # Changed to 252
        
        # Get actual test prices
        actual = y_test.tolist()
        
        # Get predicted test prices
        predicted = y_pred.tolist()
        
        # Get train prices
        train_prices = y_train.tolist()
        
        return {
            'Symbol': symbol,
            'RMSE': rmse,
            'MSE': mse,
            'MAPE': mape,
            'Predicted_Prices': predicted,
            'Actual_Prices': actual,
            'Future_Price_Predictions': future_prices,
            'Train_Prices': train_prices
        }
    
    except Exception as e:
        logger.error(f"Error processing symbol {symbol} with XGBoost: {e}")
        return {
            'Symbol': symbol,
            'RMSE': None,
            'MSE': None,
            'MAPE': None,
            'Predicted_Prices': [],
            'Actual_Prices': [],
            'Future_Price_Predictions': [],
            'Train_Prices': []
        }

# Main processing block
if __name__ == '__main__':
    # Create a list to store forecast results
    forecast_results = []

    # Parallel processing for all symbols
    num_cores = multiprocessing.cpu_count()
    results = Parallel(n_jobs=num_cores)(
        delayed(process_symbol_xgboost)(symbol) for symbol in symbols
    )

    # Add the results to forecast_results list
    forecast_results.extend(results)

    # Create the summary DataFrame
    summary_df = pd.DataFrame(forecast_results)

    # Convert lists to strings for saving
    summary_df['Predicted_Prices'] = summary_df['Predicted_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Actual_Prices'] = summary_df['Actual_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Future_Price_Predictions'] = summary_df['Future_Price_Predictions'].apply(
        lambda x: ', '.join(map(str, x))
    )
    summary_df['Train_Prices'] = summary_df['Train_Prices'].apply(
        lambda x: ', '.join(map(str, x))
    )

    # Save the summary DataFrame to CSV with the specified filename
    summary_file = os.path.join(output_folder, 'forecast_summary_XGBoost.csv')  # Updated filename
    summary_df.to_csv(summary_file, index=False)

    print(f"Tất cả kết quả đã được lưu trong {summary_file}")

    # Print the summary DataFrame
    print("\nBảng tổng hợp kết quả dự báo:")
    print(summary_df)


INFO:pinecone_plugin_interface.logging:Discovering subpackages in _NamespacePath(['C:\\Users\\Pham Ty\\AppData\\Roaming\\Python\\Python312\\site-packages\\pinecone_plugins'])
INFO:pinecone_plugin_interface.logging:Looking for plugins in pinecone_plugins.inference
INFO:pinecone_plugin_interface.logging:Installing plugin inference into Pinecone
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2
INFO:__main__:SentenceTransformer 'all-MiniLM-L6-v2' loaded successfully.


Đã có đủ dữ liệu cho tất cả 50 mã cổ phiếu.
Tất cả kết quả đã được lưu trong C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2\forecast_summary_XGBoost.csv

Bảng tổng hợp kết quả dự báo:
   Symbol       RMSE          MSE      MAPE  \
0    NVDA  43.837172  1921.697610  0.360515   
1    INTC   2.244394     5.037303  0.047655   
2    PLTR   1.360853     1.851922  0.037777   
3    TSLA  13.349218   178.201608  0.056915   
4    AAPL  20.832438   433.990474  0.066988   
5     BBD   0.496065     0.246080  0.151817   
6       T   0.682546     0.465869  0.030761   
7    SOFI   0.289385     0.083744  0.028162   
8     WBD   2.486245     6.181415  0.219172   
9    SNAP   0.640605     0.410374  0.032621   
10    NIO   0.360952     0.130286  0.038337   
11    BTG   0.083994     0.007055  0.020210   
12      F   0.309506     0.095794  0.018049   
13    AAL   0.492877     0.242927  0.027849   
14    NOK   0.109273     0.011941  0.020229   
15    BAC   0.589656     0.347695  0.013198   
16  

In [1]:
import os
import pandas as pd
import numpy as np
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.graph_objs as go
import logging
from datetime import timedelta

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s %(levelname)s:%(message)s')

# Paths to data
folder_path = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset-2'  # Folder with stock CSV files
output_folder = r'C:\Users\Pham Ty\Desktop\Thesis-Predict\Dataset_processed_2'  # Output folder

# List of stock symbols
symbols = ['NVDA', 'INTC', 'PLTR', 'TSLA', 'AAPL', 'BBD', 'T', 'SOFI',
           'WBD', 'SNAP', 'NIO', 'BTG', 'F', 'AAL', 'NOK', 'BAC',
           'CCL', 'ORCL', 'AMD', 'PFE', 'KGC', 'MARA', 'SLB', 'NU',
           'MPW', 'MU', 'LCID', 'NCLH', 'RIG', 'AMZN', 'ABEV', 'U',
           'LUMN', 'AGNC', 'VZ', 'WBA', 'WFC', 'RIVN', 'UPST', 'GRAB',
           'CSCO', 'VALE', 'AVGO', 'PBR', 'GOOGL', 'SMMT', 'GOLD',
           'CMG', 'BCS', 'UAA']

# Load forecast_summary_XGBoost.csv
forecast_summary_file = os.path.join(output_folder, 'forecast_summary_XGBoost.csv')
if os.path.exists(forecast_summary_file):
    forecast_summary_df = pd.read_csv(forecast_summary_file)
    # Parse string lists into actual lists
    for col in ['Predicted_Prices', 'Actual_Prices', 'Future_Price_Predictions', 'Train_Prices']:
        if col in forecast_summary_df.columns:
            forecast_summary_df[col] = forecast_summary_df[col].apply(
                lambda x: [float(item.strip()) for item in x.split(',')] if pd.notnull(x) else []
            )
    logging.info("Loaded 'forecast_summary_XGBoost.csv' successfully.")
else:
    forecast_summary_df = pd.DataFrame()
    logging.warning("Could not find 'forecast_summary_XGBoost.csv'. Evaluation metrics and forecasts will not be displayed.")

# Initialize Dash app
app = dash.Dash(__name__)
server = app.server  # For deployment

# Define app layout
app.layout = html.Div([
    html.H1('Stock Price Predictions Dashboard (XGBoost)'),
    html.Div([
        html.Label('Select Stock Symbol'),
        dcc.Dropdown(
            id='stock-dropdown',
            options=[{'label': s, 'value': s} for s in symbols],
            value='NVDA'  # Default stock
        )
    ], style={'width': '25%', 'display': 'inline-block'}),
    html.Div([
        html.Label('Select Date Range'),
        dcc.DatePickerRange(
            id='date-picker',
            min_date_allowed='2014-09-18',  # Adjusted min date for 10-year period
            max_date_allowed='2026-12-31',  # Extended max date to accommodate future forecasts
            start_date='2014-09-18',        # Default start date
            end_date='2025-09-18'           # Extended default end date to include 1-year forecast
        )
    ], style={'display': 'inline-block', 'marginLeft': '50px'}),
    html.Div([
        html.Label('Show 1-Year Forecast'),
        dcc.Checklist(
            id='forecast-checkbox',
            options=[{'label': 'Include 1-Year Forecast', 'value': 'show_forecast'}],
            value=[],  # Default unchecked
            inline=True
        )
    ], style={'marginTop': '20px'}),
    dcc.Graph(id='price-graph'),
    html.Div(id='metrics-output', style={'marginTop': '20px'})
])

# Define callback to update graph and evaluation metrics
@app.callback(
    [Output('price-graph', 'figure'),
     Output('metrics-output', 'children')],
    [Input('stock-dropdown', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date'),
     Input('forecast-checkbox', 'value')]
)
def update_graph(selected_stock, start_date, end_date, forecast_option):
    # Convert start_date and end_date to datetime
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)

    # Get the forecast data for the selected stock
    forecast_row = forecast_summary_df[forecast_summary_df['Symbol'] == selected_stock]

    if forecast_row.empty:
        logging.error(f"No forecast data found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"No forecast data found for {selected_stock}."

    # Parse 'Actual_Prices', 'Predicted_Prices', 'Future_Price_Predictions', 'Train_Prices'
    actual_prices = forecast_row.iloc[0]['Actual_Prices']
    predicted_prices = forecast_row.iloc[0]['Predicted_Prices']
    future_prices = forecast_row.iloc[0]['Future_Price_Predictions']
    train_prices = forecast_row.iloc[0]['Train_Prices']

    # Load the stock data to get dates
    stock_data_file = os.path.join(folder_path, f'{selected_stock}.csv')
    if not os.path.exists(stock_data_file):
        logging.error(f"Stock data file not found for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Stock data file not found for {selected_stock}."

    df_stock = pd.read_csv(stock_data_file)
    if 'Date' not in df_stock.columns or 'Close' not in df_stock.columns:
        logging.error(f"Incorrect data format in stock data file for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Incorrect data format in stock data file for {selected_stock}."

    # Convert 'Date' to datetime
    df_stock['Date'] = pd.to_datetime(df_stock['Date'])
    df_stock_sorted = df_stock.sort_values('Date')
    dates = df_stock_sorted['Date']

    # Derive split_date based on 8-year training period
    split_date = df_stock_sorted['Date'].min() + pd.DateOffset(years=8)

    # Get dates for Train and Test based on split_date
    train_mask = df_stock_sorted['Date'] <= split_date
    test_mask = df_stock_sorted['Date'] > split_date

    df_train = df_stock_sorted[train_mask]
    df_test = df_stock_sorted[test_mask]

    # Ensure that 'Actual_Prices' and 'Predicted_Prices' correspond to the test set
    test_dates = df_test['Date'].reset_index(drop=True)
    actual_prices = actual_prices[:len(test_dates)]  # Trim if necessary
    predicted_prices = predicted_prices[:len(test_dates)]

    if len(actual_prices) != len(predicted_prices):
        logging.error(f"Length mismatch between actual and predicted prices for {selected_stock}.")
        return {
            'data': [],
            'layout': go.Layout(
                title=f'{selected_stock} Price Prediction',
                xaxis={'title': 'Date'},
                yaxis={'title': 'Price'}
            )
        }, f"Length mismatch in data for {selected_stock}."

    # Create DataFrame for Train set within selected date range
    df_train_filtered = df_train[(df_train['Date'] >= start_date) & (df_train['Date'] <= end_date)]

    # Create DataFrame for Test set within selected date range
    df_test_filtered = pd.DataFrame({
        'Date': test_dates[:len(actual_prices)],
        'Actual_Price': actual_prices,
        'Predicted_Price': predicted_prices
    })
    df_test_filtered = df_test_filtered[
        (df_test_filtered['Date'] >= start_date) & (df_test_filtered['Date'] <= end_date)
    ]

    # Initialize data list for plotting
    data = []

    # Add Train Prices if available in the selected date range
    if not df_train_filtered.empty:
        data.append(
            go.Scatter(
                x=df_train_filtered['Date'],
                y=df_train_filtered['Close'],
                mode='lines',
                name='Train Price',
                line=dict(color='red')
            )
        )

    # Add Actual Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Actual_Price'],
                mode='lines',
                name='Actual Test Price',
                line=dict(color='green')
            )
        )

    # Add Predicted Test Prices
    if not df_test_filtered.empty:
        data.append(
            go.Scatter(
                x=df_test_filtered['Date'],
                y=df_test_filtered['Predicted_Price'],
                mode='lines',
                name='Predicted Test Price',
                line=dict(color='blue')
            )
        )

    # Add vertical line to indicate the split between train and test
    if pd.notnull(split_date):
        data.append(
            go.Scatter(
                x=[split_date, split_date],
                y=[df_stock_sorted['Close'].min(), df_stock_sorted['Close'].max()],
                mode='lines',
                name='Train-Test Split',
                line=dict(color='black', dash='dash')
            )
        )

    # Add future price predictions if selected
    if 'show_forecast' in forecast_option:
        # Generate future dates (252 business days)
        last_date = df_stock_sorted['Date'].max()
        future_dates = pd.bdate_range(start=last_date + pd.Timedelta(days=1), periods=len(future_prices))

        if len(future_prices) != len(future_dates):
            logging.error(f"Length mismatch between future prices ({len(future_prices)}) and future dates ({len(future_dates)}) for {selected_stock}.")
        else:
            df_future = pd.DataFrame({
                'Date': future_dates,
                'Forecasted_Price': future_prices
            })

            # Filter future forecasts based on selected date range
            df_future_filtered = df_future[
                (df_future['Date'] >= start_date) & (df_future['Date'] <= end_date)
            ]

            if not df_future_filtered.empty:
                data.append(
                    go.Scatter(
                        x=df_future_filtered['Date'],
                        y=df_future_filtered['Forecasted_Price'],
                        mode='lines',
                        name='1-Year Forecast',
                        line=dict(dash='dash', color='orange')
                    )
                )
            else:
                logging.warning(f"No forecasted data within the selected date range for {selected_stock}.")

    # Create figure with dynamic title based on selected date range
    figure = {
        'data': data,
        'layout': go.Layout(
            title=f'{selected_stock} Price Prediction ({start_date.date()} to {end_date.date()})',
            xaxis={'title': 'Date'},
            yaxis={'title': 'Price'},
            hovermode='closest'
        )
    }

    # Get RMSE, MSE, and MAPE
    rmse = forecast_row.iloc[0]['RMSE']
    mse = forecast_row.iloc[0]['MSE']
    mape = forecast_row.iloc[0]['MAPE']

    # Format metrics output
    if pd.notnull(rmse) and pd.notnull(mse) and pd.notnull(mape):
        metrics_output = [
            html.P(f'Root Mean Squared Error (RMSE): {rmse:.4f}'),
            html.P(f'Mean Squared Error (MSE): {mse:.4f}'),
            html.P(f'Mean Absolute Percentage Error (MAPE): {mape:.2%}')
        ]
    else:
        metrics_output = [html.P("Evaluation metrics are not available.")]

    return figure, metrics_output

# Run Dash app
if __name__ == '__main__':
    app.run_server(debug=False, use_reloader=False)

2024-10-21 20:51:06,508 INFO:Loaded 'forecast_summary_XGBoost.csv' successfully.
